In [ ]:
# TODO: set covariance to 0 if less than played in less than 10% of games

In [2]:
import pandas as pd
import os
os.chdir("/Users/alandu/Documents/DFS/")
# from datetime import date, timedelta
import datetime
from pathlib import Path
import numpy as np

Functions

In [36]:
def dateRange(start, end, delta):
    dates = []
    curr = start
    while curr < end:
        dates.append(curr)
        curr += delta
    dates = list(dates)
#     output = [date.strftime('%Y-%m-%d') for date in dates]
    return dates

Create first/base case hist_fpts_mat_update matrix (only need to run once at beginning of season)

In [90]:
# # first date
# date = "2017-04-02"

# results_mat = pd.read_csv("MLB/data_warehouse/" + date + "/player_results.csv")
# results_mat['Name_Team'] = results_mat['Player'] + "_" + results_mat['Team']
# results_mat = results_mat[['Name_Team', 'Actual Score']]
# results_mat.rename(columns={'Actual Score':date}, inplace = True)
# results_mat

# results_mat.to_csv("MLB/data_warehouse/" + "2017-04-03" + "/hist_fpts_mat_update.csv", index=False, na_rep="NA")

Create hist_fpts_mat_update for rest of season

In [30]:
# this code could be cleaned up. got messy because missing some date folders due to missing data / all star break

# rest of dates
date_list = dateRange(datetime.date(2017, 4, 4), datetime.date(2017, 8, 15), datetime.timedelta(days=1))

for date in date_list:
    file_path_date = Path("MLB/data_warehouse/" + date.strftime('%Y-%m-%d'))
    if file_path_date.exists():
        date_tm1 = date - datetime.timedelta(days=1)
        
        # read in hist_fpts_mat_update from date t-1
        file_path_hist = Path("MLB/data_warehouse/" + date_tm1.strftime('%Y-%m-%d') + "/hist_fpts_mat_update.csv")
        if file_path_hist.exists():
            hist_fpts_mat = pd.read_csv(str(file_path_hist))
        else:
            # fill in dates where folder is missing
            temp_date = date_tm1 - datetime.timedelta(days=1)
            file_path_hist = Path("MLB/data_warehouse/" + temp_date.strftime('%Y-%m-%d') + "/hist_fpts_mat_update.csv")
            num_missing = 1
            while (file_path_hist.exists() == False):
                temp_date = temp_date - datetime.timedelta(days=1)
                file_path_hist = Path("MLB/data_warehouse/" + temp_date.strftime('%Y-%m-%d') + "/hist_fpts_mat_update.csv")
                num_missing += 1
            hist_fpts_mat = pd.read_csv(str(file_path_hist))
            print("date(s) missing after " + date.strftime('%Y-%m-%d'))
            for i in range(0,num_missing):
                new_column = temp_date + datetime.timedelta(days=i+1)
                
                # add missing date
                new_column_m1 = new_column - datetime.timedelta(days=1)
#                 print(new_column_m1.strftime('%Y-%m-%d'))
                file_path_results = Path("MLB/data_warehouse/" + new_column_m1.strftime('%Y-%m-%d') + "/player_results.csv")
                if file_path_results.exists():
                    results_mat = pd.read_csv(str(file_path_results))
                    results_mat['Name_Team'] = results_mat['Player'] + "_" + results_mat['Team']
                    hist_fpts_mat = hist_fpts_mat.merge(results_mat[['Name_Team', 'Actual Score']], on="Name_Team", how='outer')
                    hist_fpts_mat.rename(columns={'Actual Score':new_column_m1.strftime('%Y-%m-%d')}, inplace = True)
                elif Path("MLB/data_warehouse/" + new_column_m1.strftime('%Y-%m-%d')).exists():
                    hist_fpts_mat[new_column_m1.strftime('%Y-%m-%d')] = float('NaN')
                
                # add previous dates NA
                hist_fpts_mat[new_column.strftime('%Y-%m-%d')] = float('NaN')
        
        # add fpts for date t
        file_path_results = Path("MLB/data_warehouse/" + date_tm1.strftime('%Y-%m-%d') + "/player_results.csv")
        if file_path_results.exists():
            results_mat = pd.read_csv(str(file_path_results))
            results_mat['Name_Team'] = results_mat['Player'] + "_" + results_mat['Team']

            new_hist_fpts_mat = hist_fpts_mat.merge(results_mat[['Name_Team', 'Actual Score']], on="Name_Team", how='outer')
            new_hist_fpts_mat.rename(columns={'Actual Score':date_tm1.strftime('%Y-%m-%d')}, inplace = True)
        else:
            new_hist_fpts_mat = hist_fpts_mat
            new_hist_fpts_mat[date_tm1.strftime('%Y-%m-%d')] = float('NaN')
        
        # write to file
        new_hist_fpts_mat.to_csv(str(file_path_date) + "/hist_fpts_mat_update.csv", index=False, na_rep="NA")

date missing after 2017-07-10
date missing after 2017-07-14
date missing after 2017-07-17
date missing after 2017-08-09


Create full covariance matrix for entire season

In [31]:
date_list = dateRange(datetime.date(2017, 4, 4), datetime.date(2017, 8, 15), datetime.timedelta(days=1))
for date in date_list:
    # create covariance matrix
    file_path_hist = Path("MLB/data_warehouse/" + date.strftime('%Y-%m-%d') + "/hist_fpts_mat_update.csv")
    if file_path_hist.exists():
        hist_fpts_mat_update = pd.read_csv(str(file_path_hist), header=None)
        hist_fpts_mat_update = hist_fpts_mat_update[1:] # remove first row (header)

        player_team_vec = hist_fpts_mat_update[0]
        player_team_vec = player_team_vec.reset_index(drop=True)

        hist_fpts_mat_update = hist_fpts_mat_update.drop(0, 1) # remove first column (names)
        hist_fpts_mat_update = hist_fpts_mat_update.astype(float)
        cov_mat_masked = np.ma.array(hist_fpts_mat_update, mask=np.isnan(hist_fpts_mat_update))
        cov_mat = np.ma.cov(cov_mat_masked, allow_masked=True)
        cov_mat = np.ma.getdata(cov_mat)
        
        # get list of team names in order
        column_teams = []
        for player_team in player_team_vec:
            column_teams.append(player_team.split('_')[1])

        # set covariance to 0 if not on same team
        for ind_row, player_team in enumerate(player_team_vec):
            row_team = player_team_vec[ind_row].split('_')[1]
            for ind_col, column_team in enumerate(column_teams):
                if row_team != column_team:
                    cov_mat[ind_row, ind_col] = 0.0
                    
        # TODO: set to 0 if less than played in less than 5% of games
                    
        # convert to pandas df to add column names
        cov_mat = pd.DataFrame(cov_mat)
        cov_mat.columns = player_team_vec
        
        # write to file
        cov_mat.to_csv("MLB/data_warehouse/" + date.strftime('%Y-%m-%d') + "/covariance_mat_update.csv", index=False, na_rep="NA")
        print(date.strftime('%Y-%m-%d') + " completed")

Subset covariance matrix for each contest

In [42]:
date_list = dateRange(datetime.date(2017, 7, 31), datetime.date(2017, 8, 1), datetime.timedelta(days=1))

for date in date_list:
    # read in contests.csv and subset by date
    contests_file = pd.read_csv("MLB/data_warehouse/contests.csv")
    contests_file = contests_file[(contests_file['Contest_Date'] == date.strftime('%Y-%m-%d'))]
    contests_file = contests_file.reset_index()
    
    for i in range(0, len(contests_file)):
        file_path_hist = Path("MLB/data_warehouse/" + date.strftime('%Y-%m-%d') + "/hist_fpts_mat_update.csv")
        file_path_date = Path("MLB/data_warehouse/" + date.strftime('%Y-%m-%d'))
        if file_path_hist.exists():
            # read in hist_fpts_mat_update (don't actually need this but used for convenience of column names)
            hist_fpts_mat_update = pd.read_csv(str(file_path_hist), header=None)

            # read in hitters.csv
            contest_name = contests_file['Entry_Fee'][i] + "entry_" + contests_file['Contest_Name'][i].replace(" ", "")
            file_path_hitters =  Path(str(file_path_date) + "/" + contest_name + "/hitters.csv")
            if file_path_hitters.exists():
                hitters_df = pd.read_csv(str(file_path_date) + "/" + contest_name + "/hitters.csv")

                # read in covariance_mat_update and update it
                covariance_mat_update = pd.read_csv(str(file_path_date) + "/covariance_mat_update.csv")

                # find indicies of each player in hitters_df in hist_fpts_mat_update
                hitters_name_team = hitters_df['Name'] + "_" + hitters_df['teamAbbrev']
                inds_match = []
                for name_team in hitters_name_team:
                    inds_match.append(hist_fpts_mat_update[hist_fpts_mat_update[0] == name_team].index.tolist())

                # find indicies of missing players (will be 0's in covariance matrix)
                inds_missing = []
                for i, ind in enumerate(inds_match):
                    if (len(ind) == 0):
                        inds_missing.append(i)

                # append 0's to covariance matrix
                inds_added = []
                for i in range(0,len(inds_missing)):
                    covariance_mat_update.loc[len(covariance_mat_update)] = 0
                    covariance_mat_update[len(covariance_mat_update)] = pd.Series(np.zeros(len(covariance_mat_update)), index=covariance_mat_update.index)
                    inds_added.append(len(covariance_mat_update)-1)

                # fill in inds where no match
                for i, ind in enumerate(inds_missing):
                    inds_match[ind] = [inds_added[i]]

                # unlist for numpy function
                inds_match = sum(inds_match, [])

                # update covariance matrix
                covariance_mat_update = np.array(covariance_mat_update)
                covariance_mat_update = covariance_mat_update[np.ix_(inds_match,inds_match)] # subset nxn matrix into mxm
                covariance_mat_update = pd.DataFrame(covariance_mat_update)
                covariance_mat_update.columns = hitters_name_team # set column names

                # write to file
                covariance_mat_update.to_csv("MLB/data_warehouse/" + date.strftime('%Y-%m-%d') + "/" + contest_name + "/covariance_mat_update.csv", index=False, na_rep="NA")

                # print
                print(date.strftime('%Y-%m-%d') + " " + contest_name + " completed")

2017-07-31 $6.00entry_MLB$350KSwingfortheFences[$50Kto1st] completed
2017-07-31 $444.00entry_MLB$425KGrandSlam[$100Kto1st!] completed
2017-07-31 $55.00entry_MLB$300KFullCount[$50Kto1st] completed
2017-07-31 $33.00entry_$4MFantasyBaseballWorldChampionshipQualifier#102 completed
2017-07-31 $4.00entry_MLB$40KFour-Seamer[20EntryMax] completed
2017-07-31 $25.00entry_MLB$25DoubleUp completed
2017-07-31 $5.00entry_MLB$5DoubleUp completed
2017-07-31 $5.00entry_MLB$5DoubleUp completed
2017-07-31 $2.00entry_MLB$2DoubleUp completed
2017-07-31 $3.00entry_MLB$3KMoonshot completed
2017-07-31 $2.00entry_MLB$4KFlare completed
2017-07-31 $1.00entry_MLB$10KSoloShot completed
2017-07-31 $0.25entry_MLB$2KQuarterJukebox[Just$0.25!] completed
2017-07-31 $0.10entry_MLB$3KDimeTime[Just$0.10!] completed
2017-07-31 $0.10entry_MLB$2KDimeTime[Just$0.10!] completed
2017-07-31 $0.25entry_MLB$200Jukebox(AllDay) completed
2017-07-31 $0.10entry_MLB$100DimeTime[Just$0.10!](AllDay) completed
2017-07-31 $8.00entry_MLB$15